# Install and configure LDMS-Darshan in the cloud using AWS

## AWS Instance

We are using the free-trial eligible instances **t2.micro** with an Ubuntu image and default configurations for storage. 

Click **Lauch instance** and access it through ssh connection:

![static analysis post collecting data](aws1.png)

![static analysis post collecting data](aws2.png)


# Single Node Execution

## Video tutorial - 1 node [here.](https://drive.google.com/file/d/1xFmOxJpRhOOWyEAMkv6fxEGIFoTA4_YZ/view?usp=sharing)

## Video tutorial - multinode [here.](https://drive.google.com/file/d/1kucLEIjtf3sB74HQ26iXd71TRH37eAOQ/view?usp=sharing)

## Install and configure 

Install all dependencies, LDMS and Darshan

In [ ]:
sudo apt-get install autoconf -y 
sudo apt-get install pkg-config -y
sudo apt-get update -y

sudo apt-get install hdf5-tools libhdf5-openmpi-dev openmpi-bin -y
sudo apt-get install python3.10 -y 
sudo apt-get install python-dev-is-python3 -y
sudo apt-get install make -y 
sudo apt-get install bison -y 
sudo apt-get install flex -y
sudo apt-get update -y

git clone https://github.com/ovis-hpc/ovis.git 
cd ovis && mkdir build
./autogen.sh && cd build
../configure --prefix=${HOME}/ovis/build 
make
make install

set-ldms-env.sh - Create an script to easily setup the LDMS paths and variables

In [ ]:
#!/bin/sh
export LDMS_INSTALL_PATH=/home/ubuntu/ovis/build
export LD_LIBRARY_PATH=$LDMS_INSTALL_PATH/lib/:$LD_LIBRARY_PATH
export LDMSD_PLUGIN_LIBPATH=$LDMS_INSTALL_PATH/lib/ovis-ldms
export ZAP_LIBPATH=$LDMS_INSTALL_PATH/lib/ovis-ldms
export PATH=$LDMS_INSTALL_PATH/sbin:$LDMS_INSTALL_PATH/bin:$PATH
export PYTHONPATH=/usr/local/lib/python3.10/dist-packages

export COMPONENT_ID="1"
export SAMPLE_INTERVAL="1000000"
export SAMPLE_OFFSET="0"
export HOSTNAME="localhost"

In [ ]:
cd
source set-ldms-env.sh
git clone https://github.com/darshan-hpc/darshan.git
cd darshan
mkdir build
./prepare.sh && cd build/
../configure --with-log-path=${HOME}/darshan/build/logs --prefix=${HOME}/darshan/build/install --with-jobid-env=PBS_JOBID CC=mpicc --enable-ldms-mod --with-ldms=${HOME}/ovis/build
make
make install

Darshan installation needs to return the following configuration, showing that LDMS runtime module was recognized:

```
   Darshan Runtime Version 3.4.4 configured with the following features:
           MPI C compiler                - mpicc
           GCC-compatible compiler       - yes
           NULL          module support  - yes
           POSIX         module support  - yes
           STDIO         module support  - yes
           DXT           module support  - yes
           MPI-IO        module support  - yes
           AUTOPERF MPI  module support  - no
           AUTOPERF XC   module support  - no
           HDF5          module support  - no
           PnetCDF       module support  - no
           BG/Q          module support  - no
           Lustre        module support  - no
           MDHIM         module support  - no
           HEATMAP       module support  - yes
           LDMS          runtime module  - yes
           Memory alignment in bytes     - 8
           Log file env variables        - N/A
           Location of Darshan log files - /home/ubuntu/darshan/build/logs
           Job ID env variable           - PBS_JOBID
           MPI-IO hints                  - romio_no_indep_rw=true;cb_nodes=4
```

set-ldms-darshan-env.sh - Create an script to easily setup the Darshan paths and variables:

In [ ]:
#!/bin/sh
export DARSHAN_INSTALL_PATH=/home/ubuntu/darshan/build/install/
export LD_PRELOAD=/home/ubuntu/darshan/build/install/lib/libdarshan.so
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$DARSHAN_INSTALL_PATH/lib
export DARSHAN_MOD_ENABLE="DXT_POSIX,DXT_MPIIO"

# enable LDMS data collection. No runtime data collection will occur if this is not exported.
export DARSHAN_LDMS_ENABLE=
# determine which modules we want to publish to ldmsd
export DARSHAN_LDMS_ENABLE_MPIIO=
export DARSHAN_LDMS_ENABLE_POSIX=
export DARSHAN_LDMS_ENABLE_STDIO=
export DARSHAN_LDMS_ENABLE_HDF5=
export DARSHAN_LDMS_ENABLE_ALL=
export DARSHAN_LDMS_VERBOSE=

# darshanConnector
export DARSHAN_LDMS_STREAM=darshanConnector
export DARSHAN_LDMS_XPRT=sock
export DARSHAN_LDMS_HOST=localhost
export DARSHAN_LDMS_PORT=10444
export DARSHAN_LDMS_AUTH=none

Set paths and variables for Darshan:

In [ ]:
source ./set-ldms-darshan-env.sh


darshan_stream_store.conf - Create the LDMS stream configuration file:

In [ ]:
load name=hello_sampler
config name=hello_sampler producer=${HOSTNAME} instance=${HOSTNAME}/hello_sampler stream=darshanConnector component_id=${COMPONENT_ID}
start name=hello_sampler interval=${SAMPLE_INTERVAL} offset=${SAMPLE_OFFSET}

load name=stream_csv_store
config name=stream_csv_store path=./streams/store container=csv stream=darshanConnector rolltype=3 rollover=500000

### **[TERMINAL 1]** Run the deamon:

Check the output log file to see if there is any ERROR or relevant debug message.

Check it LDMS is running/connected to the configured port.

In [ ]:
sudo chmod 1777 /var/run/
ldmsd -x sock:10444 -c darshan_stream_store.conf -l /tmp/darshan_stream_store.log -v DEBUG 
cat /tmp/darshan_stream_store.log
ldms_ls -p 10444 -x sock -v -v
ps auwx | grep ldmsd | grep -v grep

Output:

```
root@ip-172-31-35-162:/home/ubuntu# ldms_ls -p 10444 -x sock -v -v
Hostname    : localhost
IP Address  : 127.0.0.1
Port        : 10444
Transport   : sock
Schema Digest                                                    Schema         Instance                 Flags  Msize  Dsize  Hsize  UID    GID    Perm       Update            Duration          Info    
---------------------------------------------------------------- -------------- ------------------------ ------ ------ ------ ------ ------ ------ ---------- ----------------- ----------------- --------
root@ip-172-31-35-162:/home/ubuntu# ps auwx | grep ldmsd | grep -v grep
ubuntu     55777  0.0  0.5 260092  4952 ?        Ssl  01:24   0:00 ldmsd -x sock:10444 -c darshan_stream_store.conf -l /tmp/darshan_stream_store.log -v DEBUG
```

### **[TERMINAL 2]** Run the python application:

In [ ]:
source set-ldms-darshan-env.sh
export PROG=mpi-io-test
export DARSHAN_TMP=/tmp/darshan-ldms-test
export DARSHAN_TESTDIR=/home/ubuntu/darshan/darshan-test/regression
export DARSHAN_LOGFILE=$DARSHAN_TMP/${PROG}.darshan


mkdir -p $DARSHAN_TMP
cd $DARSHAN_TESTDIR
mpicc $DARSHAN_TESTDIR/test-cases/src/${PROG}.c -o $DARSHAN_TMP/${PROG}
cd $DARSHAN_TMP
./${PROG} -f $DARSHAN_TMP/${PROG}.tmp.dat

Output:

```
# Using mpi-io calls.
nr_procs = 1, nr_iter = 1, blk_sz = 16777216, coll = 0
# total_size = 16777216
# Write: min_t = 0.006564, max_t = 0.006564, mean_t = 0.006564, var_t = 0.000000
# Read:  min_t = 0.010569, max_t = 0.010569, mean_t = 0.010569, var_t = 0.000000
Write bandwidth = 2555.935759 Mbytes/sec
Read bandwidth = 1587.377742 Mbytes/sec
```

Check results collected by the LDMS stream:

In [ ]:
cat /tmp/darshan_stream_store.log

# Multiple nodes Execution

In the AWS dashboard, launch 2 instances under the same Virtual Private Cloud:

- In Network settings choose a **subnet** and add a **security group rule** using TCP protocol, source type Anywhere and open ports range for: 10000-20000
- Guarantee access between instances: create and share public keys and add to authorized_keys file using **ssh-keygen**

In [ ]:
sudo apt-get install autoconf -y 
sudo apt-get install pkg-config -y
sudo apt-get update -y

sudo apt-get install hdf5-tools libhdf5-openmpi-dev openmpi-bin -y
sudo apt-get install python3.10 -y 
sudo apt-get install python-dev-is-python3 -y
sudo apt-get install make -y 
sudo apt-get install bison -y 
sudo apt-get install flex -y
sudo apt-get update -y
sudo apt install net-tools

git clone https://github.com/ovis-hpc/ovis.git 
cd ovis && mkdir build
./autogen.sh && cd build
../configure --prefix=${HOME}/ovis/build 
make
make install

set-ldms-env.sh - Replace HOSTNAME with instance hostname that will launch LDMS

In [ ]:
#!/bin/sh
export LDMS_INSTALL_PATH=/home/ubuntu/ovis/build
export LD_LIBRARY_PATH=$LDMS_INSTALL_PATH/lib/:$LD_LIBRARY_PATH
export LDMSD_PLUGIN_LIBPATH=$LDMS_INSTALL_PATH/lib/ovis-ldms
export ZAP_LIBPATH=$LDMS_INSTALL_PATH/lib/ovis-ldms
export PATH=$LDMS_INSTALL_PATH/sbin:$LDMS_INSTALL_PATH/bin:$PATH
export PYTHONPATH=/usr/local/lib/python3.10/dist-packages

export COMPONENT_ID="1"
export SAMPLE_INTERVAL="1000000"
export SAMPLE_OFFSET="0"
export HOSTNAME="ip-172-31-35-72"

In [ ]:
cd
source set-ldms-env.sh
git clone https://github.com/darshan-hpc/darshan.git
cd darshan
mkdir build
./prepare.sh && cd build/
../configure --with-log-path=${HOME}/darshan/build/logs --prefix=${HOME}/darshan/build/install --with-jobid-env=PBS_JOBID CC=mpicc --enable-ldms-mod --with-ldms=${HOME}/ovis/build
make
make install

set-darshan-env.sh - Replace HOSTNAME with instance hostname that will launch LDMS

In [ ]:
#!/bin/sh
export DARSHAN_INSTALL_PATH=/home/ubuntu/darshan/build/install/
export LD_PRELOAD=/home/ubuntu/darshan/build/install/lib/libdarshan.so
export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$DARSHAN_INSTALL_PATH/lib
export DARSHAN_MOD_ENABLE="DXT_POSIX,DXT_MPIIO"

# enable LDMS data collection. No runtime data collection will occur if this is not exported.
export DARSHAN_LDMS_ENABLE=
# determine which modules we want to publish to ldmsd
export DARSHAN_LDMS_ENABLE_MPIIO=
export DARSHAN_LDMS_ENABLE_POSIX=
export DARSHAN_LDMS_ENABLE_STDIO=
export DARSHAN_LDMS_ENABLE_HDF5=
export DARSHAN_LDMS_ENABLE_ALL=
export DARSHAN_LDMS_VERBOSE=

# darshanConnector
export DARSHAN_LDMS_STREAM=darshanConnector
export DARSHAN_LDMS_XPRT=sock
export DARSHAN_LDMS_HOST="ip-172-31-35-72"
export DARSHAN_LDMS_PORT=10444
export DARSHAN_LDMS_AUTH=none

In [ ]:
source set-ldms-darshan-env.sh

In [ ]:
darshan_stream_store.conf

In [ ]:
load name=hello_sampler
config name=hello_sampler producer=${HOSTNAME} instance=${HOSTNAME}/hello_sampler stream=darshanConnector component_id=${COMPONENT_ID}
start name=hello_sampler interval=${SAMPLE_INTERVAL} offset=${SAMPLE_OFFSET}

load name=stream_csv_store
config name=stream_csv_store path=./streams/store container=csv stream=darshanConnector rolltype=3 rollover=500000

**Node 1 - Running LDMS-Darshan daemon:**

- Check if the server running LDMS port is open and listening:
- Check if there are firewalls blocking the access:

In [ ]:
sudo chmod 1777 /var/run/
ldmsd -x sock:10444 -c darshan_stream_store.conf -l /tmp/darshan_stream_store.log -v DEBUG 
cat /tmp/darshan_stream_store.log
ldms_ls -p 10444 -x sock -v -v
ps auwx | grep ldmsd | grep -v grep
netstat -tuln | grep 10444
sudo iptables -L OUTPUT --line-numbers

**Node 2 - Running application:**

- Run MPI application
- Check if LDMS connector is collecting data

In [ ]:
export PROG=mpi-io-test
export DARSHAN_TMP=/tmp/darshan-ldms-test
export DARSHAN_TESTDIR=/home/ubuntu/darshan/darshan-test/regression
export DARSHAN_LOGFILE=$DARSHAN_TMP/${PROG}.darshan

mkdir -p $DARSHAN_TMP
cd $DARSHAN_TESTDIR
mpicc $DARSHAN_TESTDIR/test-cases/src/${PROG}.c -o $DARSHAN_TMP/${PROG}
cd $DARSHAN_TMP
./${PROG} -f $DARSHAN_TMP/${PROG}.tmp.dat
cat /tmp/darshan_stream_store.log

CSV files located in the path configured in file darshan_stream_store.conf:

In [ ]:
ls ./streams/store/csv/